# 크롤링 예시 

In [ ]:
#'imdb.com' 에서 Movie 정보 크롤링 해오는 과정 
from bs4 import BeautifulSoup as bs
import requests

def movie_crawler(url):
    res = requests.get(url)

    # 예상치 못하게 404나 500을 받았는데, html을 파싱하려고 하면
    # 에러가 발생하므로, 200(성공)일때만 코드가 실행되도록 합니다.
    if res.status_code == 200:
        soup = bs(res.text,'html.parser')

        #
        # titles = soup.select('#main > div > div.list.detail > div > table > tbody > tr > td.overview-top > h4 > a')
        # images = soup.select('#img_primary > div > a > div > img')
        #
        # 이렇게 전체 html에서 titles(제목들), images(포스터 이미지들)를 가져오는 대신
        # 하나의 movie 정보를 감싸고 있는 div 태그를 가져온 다음
        # 그 안에 있는 포스터 이미지, 영화 제목, 평점 등을 가져오면
        # for문을 한번만 돌아도됩니다!

        # 이렇게 css selector를 가지고 받아올 수도 있고
        movies = soup.select('#main > div > div.list.detail > div')
        # 이렇게 속성을 가지고 받아올 수도 있습니다.
        movies = soup.findAll('div', {'itemtype': 'http://schema.org/Movie'})

        # title, image, running_time, score, genre, directors, actors 순으로 넣겠습니다.
        table = []
        for movie in movies:
            row = []

            title = movie.findAll('h4', {'itemprop': 'name'})
            image = movie.findAll('img', {'itemprop': 'image'})
            running_time = movie.findAll('time', {'itemprop': 'duration'})
            score = movie.select('div.rating_txt > div > strong')
            genres = movie.findAll('span', {'itemprop': 'genre'})
            directors = movie.findAll('span', {'itemprop': 'director'})
            actors = movie.findAll('span', {'itemprop': 'actors'})


            # strip()은 앞뒤 공백을 지워줍니다.
            # title가 빈 리스트([])인데, title[0]를 하면 index out of range 에러가 납니다.
            # 에러가 나는 것을 방지해주기 위해서 뒤에 len(title)이 0보다 클때만 title[0]를 하게 하고
            # 아니면 "" 빈 스트링을 row에 append 합니다.
            row.append(title[0].text.strip() if len(title)>0 else "")
            row.append(image[0]['src'].strip() if len(image)>0 else "")
            row.append(running_time[0].text.strip() if len(running_time)>0 else "")
            row.append(score[0].text.strip() if len(score)>0 else "")
            
            # genre, director, actor는 여러명인 경우가 있기 때문에 list로 받아옵니다.
            row.append([genre.text.strip() for genre in genres])
            row.append([director.text.strip() for director in directors])
            row.append([actor.text.strip() for actor in actors])

            # 예상대로라면
            # row <= [u'The Woman in Black 2: Angel of Death (2014)', 'http://ia.media-imdb.com/images/M/MV5BMTgxMjUyNTAxNF5BMl5BanBnXkFtZTgwNTk4MDUyMzE@._V1_UY209_CR0,0,140,209_AL_.jpg', u'98 min', u'42', [u'Drama', u'Horror', u'Thriller'], [u'Tom Harper'], [u'Helen McCrory', u'Jeremy Irvine', u'Phoebe Fox', u'Leanne Best']]
            # print row
    
            table.append(row)

        return table

jan_2015 = movie_crawler('http://www.imdb.com/movies-coming-soon/2015-01')
# print jan_2015[0]

print(jan_2015)


In [6]:
#{} 와 .format 메서드 쓰는 법 기억해 두기. 
#str.zfill(width) ->> zero padding 만드는 방법. str -> " " 형태의 어떤 것이든 OK. 
#zfill(width) 설정의 경우 계속 사용해봐서 익혀야 할듯. 

target_url = 'http://www.imdb.com/movies-coming-soon/{}'
movie_total = []
for i in range(1,13):
    # string.zfill(2)을 사용해보세요. zero padding이 생깁니다.
    date = "2015-" + str(i).zfill(2)
    print(target_url.format(date) + " crawling..")
    movie_total += movie_crawler(target_url.format(date))

http://www.imdb.com/movies-coming-soon/2015-01 crawling..
http://www.imdb.com/movies-coming-soon/2015-02 crawling..
http://www.imdb.com/movies-coming-soon/2015-03 crawling..
http://www.imdb.com/movies-coming-soon/2015-04 crawling..
http://www.imdb.com/movies-coming-soon/2015-05 crawling..
http://www.imdb.com/movies-coming-soon/2015-06 crawling..
http://www.imdb.com/movies-coming-soon/2015-07 crawling..
http://www.imdb.com/movies-coming-soon/2015-08 crawling..
http://www.imdb.com/movies-coming-soon/2015-09 crawling..
http://www.imdb.com/movies-coming-soon/2015-10 crawling..
http://www.imdb.com/movies-coming-soon/2015-11 crawling..
http://www.imdb.com/movies-coming-soon/2015-12 crawling..


In [9]:
print("총" + str(len(movie_total)) + "개의 영화 정보를 크롤링했습니다.")

# 3개만 출력해봅시다.
for i in range(0,3):
    print("========================================")
    print(movie_total[i])

총382개의 영화 정보를 크롤링했습니다.
['The Woman in Black 2: Angel of Death (2014)', 'https://images-na.ssl-images-amazon.com/images/M/MV5BMTgxMjUyNTAxNF5BMl5BanBnXkFtZTgwNTk4MDUyMzE@._V1_UY209_CR0,0,140,209_AL_.jpg', '98 min', '', ['Drama', 'Horror', 'Thriller'], ['Tom Harper'], ['Helen McCrory', 'Jeremy Irvine', 'Phoebe Fox', 'Leanne Best']]
['A Most Violent Year (2014)', 'https://images-na.ssl-images-amazon.com/images/M/MV5BMjE4OTY4ODg3Ml5BMl5BanBnXkFtZTgwMTI1MTg1MzE@._V1_UY209_CR0,0,140,209_AL_.jpg', '125 min', '', ['Action', 'Crime', 'Drama', 'Thriller'], ['J.C. Chandor'], ['Oscar Isaac', 'Jessica Chastain', 'David Oyelowo', 'Alessandro Nivola']]
['Leviafan (2014)', 'https://images-na.ssl-images-amazon.com/images/M/MV5BMjAwMTY3MTU0Ml5BMl5BanBnXkFtZTgwNzE0ODAwMzE@._V1_UY209_CR0,0,140,209_AL_.jpg', '140 min', '', ['Drama'], ['Andrey Zvyagintsev'], ['Aleksey Serebryakov', 'Elena Lyadova', 'Roman Madyanov', 'Vladimir Vdovichenkov']]


# csv 파일 읽고 쓰기

In [10]:
#csv 파일 만들기. -> with open('sample.csv','w') 통해서 해당 파일경로에 .csv 파일 만듦

#delimiter -> 지정하여 셀 나눔 실행 (엑셀파일에서)
#quotechar -> 쿵푸팬더의 경우 ' ' 안에 ,(comma)가 있어 '|쿵~3' (셀나눔) '2016|' 이런식으로 셀이 형성됨

#기억해야할 것:
    # csv.writer(파일이름, delimiter='', quotechar='',quoting=)
    # .writerow(['','',''])
    # .writerow 사이에 엔터(빈 행)있으면 그 행 수 만큼 엑셀에도 빈 행 생김
    # default delimiter는 ,(comma) quotechar는 "(double quote)
    

import csv

with open('sample.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    writer.writerow(['검사외전','범죄','126분'])
    
    writer.writerow(['쿵푸팬더3 (Kung Fu Panda 3, 2016)','애니메이션','95분'])


In [11]:
#csv 파일 읽는 방법
#open('파일명', 'r') -> 읽기모드로
#csv.reader() -> writer랑 같음 / quoting 제외
#csv파일에서는 ,로 셀나뉘어지고 |가 추가되어 있지만, 파이썬에서 print 될 때는 list값 제대로 출력되어 읽음 

data = []
with open('sample.csv','r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',',quotechar='|')
    for row in reader:
        data.append(row)

print(data)

[['검사외전', '범죄', '126분'], [], ['쿵푸팬더3 (Kung Fu Panda 3, 2016)', '애니메이션', '95분'], []]


# CSV 파일 읽고 쓰기 _ con'd 해보기

In [2]:
# 이번에는 delimiter를 세미콜론(';')으로 하고, quotechar는 '/'로 해봅시다. 
# 여러줄을 한 번에 쓸때는 writer.writerows 하면 됩니다. **
# remind:  코드중간 '\' 쓰면 enter 기능. 다음행에 이어서 코드 쓰기!!!!!

import csv 

data = [['검;사;외;전', '범죄','126분'], \
        ['쿵;푸;팬;더;3 (Kung Fu Panda 3, 2016)', \
         '애;니;메;이;션','95분']]

def write_csv(data, filename, delimiter=',', quotechar='"'):
    with open(filename, 'w') as csvfile:
        writer = csv.writer(csvfile, \
                           delimiter=delimiter,\
                           quotechar=quotechar)
        writer.writerows(data)
        
    return filename

write_csv(data, 'movies_sample.csv')

'movies_sample.csv'

In [3]:
def read_csv(filename, delimiter=',', quotechar='"'):
    data = []
    
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, \
                            delimiter=delimiter, \
                            quotechar=quotechar)
        for row in reader:
            data.append(row)
    
    return data

data = read_csv('movies_sample.csv')

for row in data:
    for item in row:
        print(item)

검;사;외;전
범죄
126분
쿵;푸;팬;더;3 (Kung Fu Panda 3, 2016)
애;니;메;이;션
95분
